## Notebook used to recommand a packaging material. 

In [63]:
import pandas as pd
import numpy as np
import random
import time
import math
import geopy.distance

In [64]:
import warnings
warnings.filterwarnings("ignore")

Dataset upload

In [65]:
materiaux = pd.read_csv("data/emballages.csv")

In [66]:
materiaux.head()

,Catégorie,Matière,kg équivalent CO2 / tonne,Taux de recyclage,Id matière,Recyclé,Type emballage
0,Papier,Pâte à papier primaire,297.0,62.0,1,0,Pochette
1,Papier,Pâte à papier primaire recyclée,317.0,52.0,2,1,Pochette
2,Papier,Pâte à papier Eucalyptus,440.0,61.5,3,0,Pochette
3,Papier,Pâte à papier Résineux,410.0,56.5,4,0,Pochette
4,Papier,Pâte à papier Feuillus,400.0,60.5,5,0,Pochette


In [67]:
materiaux.columns[6]

'Type emballage'

Changer unité de la mesure de carbon en kg CO2 / kg

In [68]:
materiaux['kg équivalent CO2 / tonne'] = materiaux['kg équivalent CO2 / tonne']/1000
materiaux.columns = ['Catégorie', 'Matière', 'kg CO2 / kg', 'Taux de recyclage', 'Id matière', 'Recyclé', 'Barrière à l’humidité']

In [69]:
materiaux

,Catégorie,Matière,kg CO2 / kg,Taux de recyclage,Id matière,Recyclé,Barrière à l’humidité
0,Papier,Pâte à papier primaire,0.297,62.0,1,0,Pochette
1,Papier,Pâte à papier primaire recyclée,0.317,52.0,2,1,Pochette
2,Papier,Pâte à papier Eucalyptus,0.440,61.5,3,0,Pochette
3,Papier,Pâte à papier Résineux,0.410,56.5,4,0,Pochette
4,Papier,Pâte à papier Feuillus,0.400,60.5,5,0,Pochette
...,...,...,...,...,...,...,...
149,Bois,Bois primaire,0.964,61.0,39,0,Boîte
150,Bois,Bois primaire,0.964,61.0,39,0,Palette
151,Bois,Bois primaire,0.964,61.0,39,0,Fourchette
152,Bois,Bois primaire,0.964,61.0,39,0,Couteau


In [82]:
df = materiaux
mat = ['Plastique transparent', 'Bouteille ']
x_0 = df[df['Matière'] == mat[0]][df[df.columns[6]] == mat[1]]

In [83]:
x_0

,Catégorie,Matière,kg CO2 / kg,Taux de recyclage,Id matière,Recyclé,Barrière à l’humidité
86,Plastique,Plastique transparent,2.35,61.0,12,0,Bouteille


Créer un score comparatif entre 0.5 et 1.5 par rapport aux emballages initiaux 

In [105]:
def recommandation(df, emballage_init):
    n = len(emballage_init)
    
    reco = [0,0]
    i = 0
    
    for mat in emballage_init: 
        #définir le materiau initial qu'on veut remplacer
        x_0 = df[df['Matière'] == mat[0]][df[df.columns[6]] == mat[1]].reset_index(drop=True)
        #choisir fermes qui produisent le meme produit uniquement
        sub_df = df[df[df.columns[6]] == mat[1]]

        num_columns = ['kg CO2 / kg', 'Taux de recyclage']
        cat_columns = ['Recyclé']
    
    #    delta_num = pd.DataFrame([])
     #   for c in num_columns:
            #mesurer les deltas en pourcentage pour les colonnes numériques
      #      delta_num[c] = sub_df[c].apply(lambda x: (x - x_0[c])/x_0[c])

   #     delta_cat = pd.DataFrame([])
    #    for c in cat_columns:
            #mesurer les deltas net pour les colonnes catégoriques (0 et 1 des labels)
     #       delta_cat[c] = sub_df[c].apply(lambda x: (x - x_0[c]))
        
        sub_df['CO2 norm'] = sub_df['kg CO2 / kg'].apply(lambda x: (x- min(sub_df['kg CO2 / kg']))/(max(sub_df['kg CO2 / kg'])-min(sub_df['kg CO2 / kg'])))                                                                                                                         
        sub_df['score'] = 0.7 - ((sub_df['CO2 norm']) - 0.2 * sub_df['Taux de recyclage']/100 - 0.1 * sub_df['Recyclé'])
    
        # définir la formule du score    
        #sub_df['% CO2 reco vs initial'] = 1 + delta_num['kg CO2 / kg']
        #sub_df['score'] = 1 + 0.8 * (delta_num['kg CO2 / kg']) + 0.1 * (delta_num['Taux de recyclage']) - 0.1 * delta_cat['Recyclé']
    
        sub_df = sub_df.sort_values('score', ascending = False)
        reco[i] = sub_df
        i = i+1
        
    
    return reco

In [106]:
materiaux.columns

Index(['Catégorie', 'Matière', 'kg CO2 / kg', 'Taux de recyclage',
       'Id matière', 'Recyclé', 'Barrière à l’humidité'],
      dtype='object')

Définir emballage initial

In [107]:
emballage_init = [['Plastique transparent', 'Bouteille '], ['Pâte à papier Eucalyptus', 'Étiquette']]

In [108]:
reco = recommandation(materiaux, emballage_init)

In [111]:
reco[0].iloc[:10,:]

,Catégorie,Matière,kg CO2 / kg,Taux de recyclage,Id matière,Recyclé,Barrière à l’humidité,CO2 norm,score
89,Plastique,Plastique 100% ex-recyclé,0.202,36.0,13,1,Bouteille,0.000000,0.872000
87,Plastique,Plastique blanc,2.025,63.0,11,0,Bouteille,0.848696,-0.022696
88,Plastique,Plastique,2.150,59.0,12,0,Bouteille,0.906890,-0.088890
86,Plastique,Plastique transparent,2.350,61.0,12,0,Bouteille,1.000000,-0.178000


In [112]:
reco[1].iloc[:10,:]

,Catégorie,Matière,kg CO2 / kg,Taux de recyclage,Id matière,Recyclé,Barrière à l’humidité,CO2 norm,score
15,Papier,Pâte à papier primaire recyclée,0.317,52.0,2,1,Étiquette,0.032154,0.871846
14,Papier,Pâte à papier primaire,0.297,62.0,1,0,Étiquette,0.000000,0.824000
18,Papier,Pâte à papier Feuillus,0.400,60.5,5,0,Étiquette,0.165595,0.655405
17,Papier,Pâte à papier Résineux,0.410,56.5,4,0,Étiquette,0.181672,0.631328
19,Papier,Pâte à papier Papier recyclé,0.470,54.0,6,1,Étiquette,0.278135,0.629865
16,Papier,Pâte à papier Eucalyptus,0.440,61.5,3,0,Étiquette,0.229904,0.593096
20,Papier,Papier moyen,0.919,61.0,7,0,Étiquette,1.000000,-0.178000


In [ ]:
def recommender_packaging(data, produit, nom_ferme = None, metric = 'score'):
    
    if nom_ferme is None : 
        df_resultat = calcul_score_sans_ferme(data, produit).sort_values(by = metric)
        resultat = df_resultat.head(10)
    else : 
        df_resultat = calcul_score(data, nom_ferme, produit).sort_values(by = metric)
        resultat = df_resultat.head(10)
    
    return resultat